In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore

pd.options.mode.chained_assignment = None  # default='warn'




df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])
cars = df["name"]
nonna = df[df["horsepower"].notna()]
na = df[df["horsepower"].isnull()]
X = nonna[["cylinders", "displacement", "acceleration"]]
y = nonna["horsepower"]
reg = LinearRegression().fit(X, y)
na["horsepower"] = reg.predict(na[["cylinders", "displacement", "acceleration"]])
df = pd.concat([nonna, na], axis=0)
df = df.reindex(np.random.permutation(df.index))
or_dum = pd.get_dummies(df["origin"], prefix="origin_")
df = pd.concat([df, or_dum], axis=1)
df.drop("origin", inplace=True, axis=1)
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin__1', "origin__2", "origin__3"]].values
y = df['mpg'].values 
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=2,epochs=200)

Train on 398 samples
Epoch 1/200
398/398 - 0s - loss: 14970.4218
Epoch 2/200
398/398 - 0s - loss: 2829.0090
Epoch 3/200
398/398 - 0s - loss: 582.1667
Epoch 4/200
398/398 - 0s - loss: 304.1420
Epoch 5/200
398/398 - 0s - loss: 241.7409
Epoch 6/200
398/398 - 0s - loss: 201.8567
Epoch 7/200
398/398 - 0s - loss: 185.8058
Epoch 8/200
398/398 - 0s - loss: 182.2459
Epoch 9/200
398/398 - 0s - loss: 176.5233
Epoch 10/200
398/398 - 0s - loss: 170.7449
Epoch 11/200
398/398 - 0s - loss: 165.6348
Epoch 12/200
398/398 - 0s - loss: 160.9796
Epoch 13/200
398/398 - 0s - loss: 156.9334
Epoch 14/200
398/398 - 0s - loss: 152.0217
Epoch 15/200
398/398 - 0s - loss: 147.2682
Epoch 16/200
398/398 - 0s - loss: 141.8555
Epoch 17/200
398/398 - 0s - loss: 137.5396
Epoch 18/200
398/398 - 0s - loss: 133.4257
Epoch 19/200
398/398 - 0s - loss: 128.3601
Epoch 20/200
398/398 - 0s - loss: 125.2091
Epoch 21/200
398/398 - 0s - loss: 124.6657
Epoch 22/200
398/398 - 0s - loss: 118.9631
Epoch 23/200
398/398 - 0s - loss: 110.3

Epoch 193/200
398/398 - 0s - loss: 12.0405
Epoch 194/200
398/398 - 0s - loss: 12.2162
Epoch 195/200
398/398 - 0s - loss: 13.4715
Epoch 196/200
398/398 - 0s - loss: 12.2630
Epoch 197/200
398/398 - 0s - loss: 11.8236
Epoch 198/200
398/398 - 0s - loss: 13.4316
Epoch 199/200
398/398 - 0s - loss: 12.5839
Epoch 200/200
398/398 - 0s - loss: 12.4532


In [11]:
pred = model.predict(x)
save_path = "./saves"
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"Before save score (RMSE): {score}")

# save neural network structure to JSON (no weights)
model_json = model.to_json()
with open(os.path.join(save_path,"3.3network.json"), "w") as json_file:
    json_file.write(model_json)

# save neural network structure to YAML (no weights)
model_yaml = model.to_yaml()
with open(os.path.join(save_path,"3.3network.yaml"), "w") as yaml_file:
    yaml_file.write(model_yaml)

# save entire network to HDF5 (save everything, suggested)
model.save(os.path.join(save_path,"3.3network.h5"))

Before save score (RMSE): 3.4632924418763946


In [13]:
from tensorflow.keras.models import load_model
model2 = load_model(os.path.join(save_path,"3.3network.h5"))
pred = model2.predict(x)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"After load score (RMSE): {score}")

After load score (RMSE): 3.4632924418763946
